### aim: 

### date: 

In [110]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [111]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [112]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [113]:
scendict = {
    '1A': {
        'hist_str': 'bc370',
        'fut_str': 'be682',
        'name': 'HIST.OZONE \n LOW TEMP.',
        'name2':'1A: NatlOzone-SSP126',
        'color':'#E8D215',
        'color2':'orange'},
    '1B': {
        'hist_str': 'bc370',
        'fut_str': 'ce417',
        'name': 'HIST. OZONE \n HIGH TEMP.',
        'name2':'1B: NatlOzone-SSP370',
        'color':'#87800A',
        'color2':'orangered'},
    '2A': {
        'hist_str': 'cj198',
        'fut_str': 'cj880',
        'name': 'FIXED OZONE \n LOW TEMP.',
        'name2':'2A: Ozone1950-SSP126',
        'color':'#2DC18E',
        'color2':'mediumseagreen'},
    '2B': {
        'hist_str': 'cj198',
        'fut_str': 'cj881',
        'name': 'FIXED OZONE \n HIGH TEMP.',
        'name2':'2B: Ozone1950-SSP370',
        'color':'#18765C',
        'color2':'green'},
    '3A': {
        'hist_str': 'cj200',
        'fut_str': 'cj484',
        'name': '1990 OZONE \n LOW TEMP.',
        'name2':'3A: Ozone1990-SSP126',
        'color':'#FF462B',
        'color2':'dodgerblue'},
    '3B': {
        'hist_str': 'cj200',
        'fut_str': 'cj504',
        'name': '1990 OZONE \n HIGH TEMP.',
        'name2':'3B: Ozone1990-SSP370',
        'color':'#822722',
        'color2':'mediumblue'}
}


def make_yearfile(td, yr, filename):
    times = pd.date_range(f"{yr}/01/15",f"{yr+1}/01/15",freq='M',closed='left')

    stor_overturning = np.zeros([12,75,332,1])
    
    for i in range(0,12):
        w = xr.open_dataset(td[i])
        stor_overturning[i,:,:,:] = w.zomsfglo.values

    data_vars = {'zomsfglo':(['time_counter', 'depthw', 'y', 'x'], stor_overturning,
    {'units': 'Sv',
    'long_name':'depth-coordinates overturning'}),
    }

    coords = {'time_counter': (['time_counter'], times),
     #       'time_centered': (['time_centered'], times),
    'nav_lat': (['y','x'], w.nav_lat),
    'nav_lon': (['y','x'], w.nav_lon),
    'depthw': (['depthw'], w.depthw)}

    attrs = {'made in':'overturning/makeyearfiles_overturning.ipynb',
    'desc': 'concatenate monthly overturning output into yearly and give good dates for later processing'
    }

    savenam = filename
    ds = xr.Dataset(data_vars=data_vars,
    coords=coords,
    attrs=attrs)
    ds.to_netcdf(savenam)

def make_yearfiles_ukesm(yrst, yren, tscen):
    print(f'SCENARIO {tscen}')
    dslist = []

    for y in range(yrst,yren):
        if ((y<1990) & ((tscen == '3A') | (tscen == '3B'))):
            tstr = scendict['1A']['hist_str']
        elif y<2015:
            tstr = scendict[tscen]['hist_str']

        else:
            tstr = scendict[tscen]['fut_str']

        try:

            td = glob.glob(f'/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_moc/nemo_{tstr}o_1m_{y}*_moc.nc')
            if (len(td) != 12):
                print(f'SCENARIO {tscen}, {y}, something missing')
            else:
                td = sorted(td)
                filename = f'/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_moc_byyear//nemo_{tstr}o_1y_{y}_moc.nc'
                try:
                    make_yearfile(td, y, filename)
                    print(f'SCENARIO {tscen}, {y}, made yearly file')
                except:
                    print(f'SCENARIO {tscen}, {y}, yearly file  not made but all monthly files present')

            #dslist.append(td[0])
        except:
            pass
            #print(f'FAIL {tscen},{tstr}, {y}')
    return td

td =make_yearfiles_ukesm(1950, 2100, '1A')
td =make_yearfiles_ukesm(1950, 2100, '1B')
td =make_yearfiles_ukesm(1950, 2100, '2A')
td =make_yearfiles_ukesm(1950, 2100, '2B')
td =make_yearfiles_ukesm(1950, 2100, '3A')
td =make_yearfiles_ukesm(1950, 2100, '3B')

SCENARIO 1A
SCENARIO 1A, 1950, made yearly file
SCENARIO 1A, 1951, made yearly file
SCENARIO 1A, 1952, made yearly file
SCENARIO 1A, 1953, made yearly file
SCENARIO 1A, 1954, made yearly file
SCENARIO 1A, 1955, made yearly file
SCENARIO 1A, 1956, made yearly file
SCENARIO 1A, 1957, made yearly file
SCENARIO 1A, 1958, made yearly file
SCENARIO 1A, 1959, made yearly file
SCENARIO 1A, 1960, made yearly file
SCENARIO 1A, 1961, made yearly file
SCENARIO 1A, 1962, made yearly file
SCENARIO 1A, 1963, made yearly file
SCENARIO 1A, 1964, made yearly file
SCENARIO 1A, 1965, made yearly file
SCENARIO 1A, 1966, made yearly file
SCENARIO 1A, 1967, made yearly file
SCENARIO 1A, 1968, made yearly file
SCENARIO 1A, 1969, made yearly file
SCENARIO 1A, 1970, made yearly file
SCENARIO 1A, 1971, made yearly file
SCENARIO 1A, 1972, made yearly file
SCENARIO 1A, 1973, made yearly file
SCENARIO 1A, 1974, made yearly file
SCENARIO 1A, 1975, made yearly file
SCENARIO 1A, 1976, made yearly file
SCENARIO 1A, 197

In [114]:
td =make_yearlist_ukesm(1950, 1951, '1A')
print(td)


yr = 1950

filename = f'/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_moc_byyear//nemo_{tstr}o_1y_{yr}_moc.nc'


NameError: name 'make_yearlist_ukesm' is not defined

In [ ]:
times = pd.date_range(f"{yr}/01/01",f"{yr+1}/01/01",freq='MS',closed='left')

print(times)

In [ ]:
w = xr.open_dataset('test2.nc')
for i in range(0,12):
    plt.figure()
    xr.plot.pcolormesh(w.zomsfglo.isel(time_counter=i).isel(x=0))